# Importing the Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Reading csv file using pandas.read_csv and saved in data variable

In [2]:
data = pd.read_csv("insurance_pre.csv")

In [3]:
data

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520
...,...,...,...,...,...,...
1333,50,male,30.970,3,no,10600.54830
1334,18,female,31.920,0,no,2205.98080
1335,18,female,36.850,0,no,1629.83350
1336,21,female,25.800,0,no,2007.94500


# Converting Categorical to Numerical

In [4]:
data=pd.get_dummies(data,drop_first=True)

In [5]:
data

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27.900,0,16884.92400,0,1
1,18,33.770,1,1725.55230,1,0
2,28,33.000,3,4449.46200,1,0
3,33,22.705,0,21984.47061,1,0
4,32,28.880,0,3866.85520,1,0
...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,1,0
1334,18,31.920,0,2205.98080,0,0
1335,18,36.850,0,1629.83350,0,0
1336,21,25.800,0,2007.94500,0,0


# Independent variable is an input
# Depedent variable is output

In [6]:
independent = data[["age","bmi","children","sex_male","smoker_yes"]]
dependent = data[["charges"]]

# Split into training set and test

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(independent,dependent, test_size = 1/3, random_state = 0)

# Standardization is changing every colume into same range of value

In [8]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [9]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
param_grid = {"criterion":["mae","mse","friedman_mse"],
             "n_estimators":[10,100],"max_features":["auto","sqrt","log2"]}

grid = GridSearchCV(RandomForestRegressor(), param_grid, refit=True, verbose = 3,n_jobs=-1)
grid.fit(X_train, y_train)

C:\Users\vijai\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Fitting 3 folds for each of 18 candidates, totalling 54 fits


[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:   14.9s
[Parallel(n_jobs=-1)]: Done  54 out of  54 | elapsed:   17.5s finished
C:\Users\vijai\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:715: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators='warn', n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid={'criterion': ['mae', 'mse', 

# Testinput is predicted by using grid predict function and saved into grid_predictions variable

In [10]:
re=grid.cv_results_
grid_predictions=grid.predict(X_test)

# Calculating evaluation metric value

In [11]:
from sklearn.metrics import r2_score
r_score=r2_score(y_test,grid_predictions)

# Best parameter

In [12]:
print("The R_score value for best parameter{}:".format(grid.best_params_),r_score)

The R_score value for best parameter{'criterion': 'mae', 'max_features': 'sqrt', 'n_estimators': 100}: 0.8721912663975724


In [13]:
table=pd.DataFrame.from_dict(re)

In [14]:
table[["param_criterion","param_max_features","param_n_estimators","mean_test_score","rank_test_score"]]

,param_criterion,param_max_features,param_n_estimators,mean_test_score,rank_test_score
0,mae,auto,10,0.781087,13
1,mae,auto,100,0.788227,9
2,mae,sqrt,10,0.773316,17
3,mae,sqrt,100,0.802530,1
4,mae,log2,10,0.766252,18
5,mae,log2,100,0.799578,5
6,mse,auto,10,0.784461,11
7,mse,auto,100,0.793060,8
8,mse,sqrt,10,0.778826,15
9,mse,sqrt,100,0.800139,3


In [15]:
age_input,bmi_input,childern_input,sex_male,smoker_yes=int(input("Enter the Age: ")),int(input("Enter the BMI: ")),int(input("Enter the Children: ")),int(input("Enter the Sex Male 0 or Female 1: ")),int(input("Enter Smoker YES 0 or NO 1: "))

Enter the Age: 24
Enter the BMI: 34
Enter the Children: 2
Enter the Sex Male 0 or Female 1: 0
Enter Smoker YES 0 or NO 1: 1


# Calculating prediction value

In [17]:
Future_Prediction=grid.predict([[age_input,bmi_input,childern_input,sex_male,smoker_yes]])
print("Future_Prediction={}".format(Future_Prediction))

Future_Prediction=[44846.5953815]
